In [1]:
from Simulation.mpc import *
from Simulation.systemFunctions import DistillationColumnAspen
from utils.helpers import *

## Initialize the system

In [2]:
# System and Snapshot paths
path = r"C:/Users\HAMEDI\Desktop\FinalDocuments\FinalDocuments\C2SplitterControlFiles\AspenFiles\dynsim\Plant\C2S_SS_simulation9.dynf"
path_snaps = r"C:/Users\HAMEDI\Desktop\FinalDocuments\FinalDocuments\C2SplitterControlFiles\AspenFiles\dynsim\Plant\AM_C2S_SS_simulation9"

In [3]:
# First initiate the system
# Nominal Conditions
nominal_conditions =  np.array([1.50032484e+05, -2.10309105e+01, 2.08083248e+01, 6.30485237e-01, 3.69514734e-01, -2.40000000e+01])

# Steady State inputs
ss_inputs = np.array([320000.0, 110.0])

# Sampling time of the system
delta_t = 1 / 6 # 10 mins

In [4]:
# steady state values
dl = DistillationColumnAspen(path, ss_inputs, nominal_conditions)
steady_states={"ss_inputs":dl.ss_inputs,
               "y_ss":dl.y_ss}
print(steady_states)
dl.close(path_snaps)

Initialization has been completed
Steady State has been completed
System is in open loop condition now!
Steady state reached!
{'ss_inputs': array([3.2e+05, 1.1e+02]), 'y_ss': array([  0.1003974 , -22.95514339])}
Deleted the last created snapshot starting with 'snp': C:/Users\HAMEDI\Desktop\FinalDocuments\FinalDocuments\C2SplitterControlFiles\AspenFiles\dynsim\Plant\AM_C2S_SS_simulation9\snpA0000.snp
Deleted the last created snapshot starting with 'snp': C:/Users\HAMEDI\Desktop\FinalDocuments\FinalDocuments\C2SplitterControlFiles\AspenFiles\dynsim\Plant\AM_C2S_SS_simulation9\snpA0001.snp


## Loading the system matrices, min max scaling, and min max of the states

In [8]:
dir_path = os.path.join(os.getcwd(), "Data/models")

In [9]:
# Defining the range of setpoints for data generation
setpoint_y = np.array([[0.002, -26.0],
                       [0.05, -16.0]])
u_min = np.array([300000.0, 100.0])
u_max = np.array([460000, 150.0])

system_data = load_and_prepare_system_data(steady_states=steady_states, setpoint_y=setpoint_y, u_min=u_min, u_max=u_max)

In [10]:
A_aug = system_data["A_aug"]
B_aug = system_data["B_aug"]
C_aug = system_data["C_aug"]

In [11]:
data_min = system_data["data_min"]
data_max = system_data["data_max"]

In [12]:
min_max_states = system_data["min_max_states"]

In [13]:
y_sp_scaled_deviation = system_data["y_sp_scaled_deviation"]

In [14]:
b_min = system_data["b_min"]
b_max = system_data["b_max"]

In [15]:
min_max_dict = system_data["min_max_dict"]

In [16]:
# Setpoints in deviation form
inputs_number = int(B_aug.shape[1])
y_sp_scenario = np.array([[0.013, -23.],
                         [0.018, -22.]])

y_sp_scenario = (apply_min_max(y_sp_scenario, data_min[inputs_number:], data_max[inputs_number:])
                 - apply_min_max(steady_states["y_ss"], data_min[inputs_number:], data_max[inputs_number:]))

n_tests = 200
set_points_len = 200
TEST_CYCLE = [False, False, False, False, False]
warm_start = 5
ACTOR_FREEZE = 4 * set_points_len
warm_start_plot = warm_start * 2 * set_points_len + ACTOR_FREEZE

In [19]:
# # # Observer Gain
# poles = np.array([0.032, 0.03501095, 0.04099389, 0.04190188, 0.07477281,
#                   0.01153274, 0.41036367])
# L = compute_observer_gain(A_aug, C_aug, poles)
# L
# # Observer Gain
poles = np.array([0.032, 0.03501095, 0.04099389, 0.04190188, 0.07477281,
                  0.5153274, 0.61036367])
# poles = np.array([0.6, 0.6, 0.55, 0.5, 0.5, 0.98, 0.95])
L = compute_observer_gain(A_aug, C_aug, poles)
L

The system is observable.


array([[ 2.61314809e-02, -7.33441466e-04],
       [ 9.57826516e-03,  2.16807172e-01],
       [-6.17077864e-01, -3.83532624e-02],
       [-5.94942729e-04, -5.93262013e-02],
       [-9.27639419e-03, -2.11065507e-01],
       [ 2.13090070e+00,  5.59402572e-02],
       [ 2.09290486e-02,  1.65750104e+00]])

## Setting The hyperparameters for the TD3 Agent

In [20]:
from TD3Agent.agent import TD3Agent
import torch

In [21]:
set_points_number = int(C_aug.shape[0])
STATE_DIM = int(A_aug.shape[0]) + set_points_number + inputs_number
ACTION_DIM = int(B_aug.shape[1])
n_outputs = C_aug.shape[0]
ACTOR_LAYER_SIZES = [256, 256, 256]
CRITIC_LAYER_SIZES = [256, 256, 256]
BUFFER_CAPACITY = 5_000_000
ACTOR_LR = 1e-4
CRITIC_LR = 1e-4
SMOOTHING_STD = 0.0001
NOISE_CLIP = 0.2
EXPLORATION_NOISE_STD = 0.05
GAMMA = 0.995
TAU = 0.005
MAX_ACTION = 1
POLICY_DELAY = 2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128
STD_START = 0.005
STD_END = 0.0
STD_DECAY_RATE = 0.99995
STD_DECAY_MODE = "exp"

In [22]:
td3_agent = TD3Agent(
    state_dim=STATE_DIM,
    action_dim=ACTION_DIM,
    actor_hidden=ACTOR_LAYER_SIZES,
    critic_hidden=CRITIC_LAYER_SIZES,
    gamma=GAMMA,
    actor_lr=ACTOR_LR,
    critic_lr=CRITIC_LR,
    batch_size=BATCH_SIZE,
    policy_delay=POLICY_DELAY,
    target_policy_smoothing_noise_std=SMOOTHING_STD,
    noise_clip=NOISE_CLIP,
    max_action=MAX_ACTION,
    tau=TAU,
    std_start=STD_START,
    std_end=STD_END,
    std_decay_rate=STD_DECAY_RATE,
    std_decay_mode=STD_DECAY_MODE,
    buffer_size=BUFFER_CAPACITY,
    device=DEVICE,
    actor_freeze=ACTOR_FREEZE,
    mode="mpc"
    )

# Buffer filling

In [23]:
from BasicFunctions.td3_functions import filling_the_buffer, add_steady_state_samples

In [24]:
# MPC parameters
predict_h = 6
cont_h = 3
b1 = (b_min[0], b_max[0])
b2 = (b_min[1], b_max[1])
bnds = (b1, b2)*cont_h
cons = []
IC_opt = np.zeros(inputs_number*cont_h)
Q1_penalty = 1.
Q2_penalty = 1.
R1_penalty = 1.
R2_penalty = 1.
Q_penalty = np.array([[Q1_penalty, 0], [0, Q2_penalty]])
R_penalty = np.array([[R1_penalty, 0], [0, R2_penalty]])

In [25]:
MPC_obj = MpcSolver(A_aug, B_aug, C_aug,
                    Q1_penalty, Q2_penalty, R1_penalty, R2_penalty,
                    predict_h, cont_h)

In [26]:
steady_states_samples_number = 100000
mpc_pretrain_samples_numbers = BUFFER_CAPACITY - steady_states_samples_number

In [27]:
filling_the_buffer(
        min_max_dict,
        A_aug, B_aug, C_aug,
        MPC_obj,
        mpc_pretrain_samples_numbers,
        Q_penalty, R_penalty,
        td3_agent,
        IC_opt, bnds, cons, chunk_size= 100000)

Processing chunk 1/49
Processing chunk 2/49
Processing chunk 3/49
Processing chunk 4/49
Processing chunk 5/49
Processing chunk 6/49
Processing chunk 7/49
Processing chunk 8/49
Processing chunk 9/49
Processing chunk 10/49
Processing chunk 11/49
Processing chunk 12/49
Processing chunk 13/49
Processing chunk 14/49
Processing chunk 15/49
Processing chunk 16/49
Processing chunk 17/49
Processing chunk 18/49
Processing chunk 19/49
Processing chunk 20/49
Processing chunk 21/49
Processing chunk 22/49
Processing chunk 23/49
Processing chunk 24/49
Processing chunk 25/49
Processing chunk 26/49
Processing chunk 27/49
Processing chunk 28/49
Processing chunk 29/49
Processing chunk 30/49
Processing chunk 31/49
Processing chunk 32/49
Processing chunk 33/49
Processing chunk 34/49
Processing chunk 35/49
Processing chunk 36/49
Processing chunk 37/49
Processing chunk 38/49
Processing chunk 39/49
Processing chunk 40/49
Processing chunk 41/49
Processing chunk 42/49
Processing chunk 43/49
Processing chunk 44/

In [28]:
add_steady_state_samples(
        min_max_dict,
        A_aug, B_aug, C_aug,
        MPC_obj,
        steady_states_samples_number,
        Q_penalty, R_penalty,
        td3_agent,
        IC_opt, bnds, cons, chunk_size= 100000)

Processing chunk 1/1
Replay buffer has been filled up with the steady_state values.


## Pre training the Agent

In [38]:
for g in td3_agent.actor_optimizer.param_groups:
    g['lr'] = 1e-7
    print(g["lr"])
# for g in td3_agent.critic_optimizer.param_groups:
#     g['lr'] = 1e-6
#     print(g["lr"])

1e-07


In [39]:
td3_agent.pretrain_from_buffer(
    num_updates=2000000,
    use_target_noise=False,
    log_interval=2000,
)

[pretrain] it=2000  bc=5.1728e-08  q=2.0739e-02
[pretrain] it=4000  bc=4.4209e-08  q=1.7298e-02
[pretrain] it=6000  bc=4.2485e-08  q=1.2966e-02
[pretrain] it=8000  bc=4.6705e-08  q=1.6945e-02
[pretrain] it=10000  bc=4.2245e-08  q=1.0121e-02
[pretrain] it=12000  bc=3.5913e-08  q=2.7244e-02
[pretrain] it=14000  bc=4.2993e-08  q=1.2925e-02
[pretrain] it=16000  bc=3.8149e-08  q=1.3498e-02
[pretrain] it=18000  bc=4.6695e-08  q=1.9241e-02
[pretrain] it=20000  bc=6.9092e-08  q=1.1548e-02
[pretrain] it=22000  bc=4.6103e-08  q=1.9344e-02
[pretrain] it=24000  bc=4.6646e-08  q=1.4001e-02
[pretrain] it=26000  bc=3.8950e-08  q=9.6068e-03
[pretrain] it=28000  bc=5.0573e-08  q=2.0976e-02
[pretrain] it=30000  bc=6.0785e-08  q=1.6000e-02
[pretrain] it=32000  bc=3.5632e-08  q=2.9258e-02
[pretrain] it=34000  bc=5.1813e-08  q=9.6079e-03
[pretrain] it=36000  bc=6.7039e-08  q=2.0233e-02
[pretrain] it=38000  bc=5.5530e-08  q=1.6620e-02
[pretrain] it=40000  bc=4.2733e-08  q=1.0942e-02
[pretrain] it=42000  bc=

## Saving and loading the agent to make sure the agent has been stored

In [40]:
filename_agent = td3_agent.save(dir_path)

Saved TD3 checkpoint to: C:\Users\HAMEDI\Desktop\DistillRL\Data/models\td3_20251109_192343.pkl


## Checking the accuracy of the agent and compare it to the MPC actions

In [41]:
import torch
import numpy as np

@torch.no_grad()
def print_accuracy(agent, n_samples: int = 10_000):
    """
    Simple BC accuracy: sample from agent.buffer, compare actor(s) to dataset actions.
    Prints overall R^2 and per-action-dimension R^2.
    """
    device = getattr(agent, "device", torch.device("cpu"))
    if len(agent.buffer) == 0:
        print("Buffer is empty; cannot evaluate.")
        return

    B = min(n_samples, len(agent.buffer))
    sample = agent.buffer.sample(B, device=device)

    # Handle ReplayBuffer (5-tuple) or PER buffer (7-tuple)
    if len(sample) == 5:
        s, a, _, _, _ = sample
    else:
        s, a, _, _, _, _, _ = sample

    s = s.to(device).float()
    a = a.to(device).float()
    if a.ndim == 1:
        a = a.unsqueeze(-1)  # [B] -> [B,1]

    # Actor predictions
    pred = agent.actor(s).clamp(-agent.max_action, agent.max_action)

    # To numpy
    y_true = a.detach().cpu().numpy()
    y_pred = pred.detach().cpu().numpy()

    # Basic sanity
    if y_true.shape != y_pred.shape:
        raise ValueError(f"Shape mismatch: true {y_true.shape} vs pred {y_pred.shape}. "
                         "Check ACTION_DIM and how actions are stored.")

    # R^2 per dimension and mean
    eps = 1e-12
    num = np.sum((y_true - y_pred) ** 2, axis=0)                              # SSE
    den = np.sum((y_true - y_true.mean(axis=0, keepdims=True)) ** 2, axis=0)  # SST
    r2_per_dim = 1.0 - num / np.maximum(den, eps)
    r2_mean = float(np.mean(r2_per_dim))

    print(f"Agent R^2 vs MPC (mean over {y_true.shape[1]} actions) : {r2_mean:.6f}")
    for j, r2j in enumerate(r2_per_dim):
        print(f"  - R^2(action[{j}]): {r2j:.6f}")

In [43]:
print_accuracy(td3_agent, n_samples=5)

Agent R^2 vs MPC (mean over 2 actions) : 1.000000
  - R^2(action[0]): 1.000000
  - R^2(action[1]): 1.000000
